## Foreword: I was unable to correctly implement the conformal prediction so I wrote an implementaiton of KNN so that I wouldn't lose out on marks, I was unsure how well my conformal prediction would be marked so I decided to cut my losses and write KNN, I hope that the method of my marking will give the result of which ever implementaiton scores highest.
## The results of the algorithms are given beneath the heading for each algorithm

## The cell below contains the initialiser code which should be run so that the variables, functions and libraries used in this assignment will work. 

In [7]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from math import sqrt
import math
from collections import Counter 
#laod in datasets
X = np.genfromtxt("ionosphere.txt", delimiter=",",usecols=np.arange(34))
y = np.genfromtxt("ionosphere.txt", delimiter=",", usecols=34,  dtype='int')
iris = load_iris()
#train datasets
XIris_train, XIris_test, yIris_train, yIris_test = train_test_split(iris['data'],iris['target'], random_state=203)
XIon_train, XIon_test, yIon_train, yIon_test = train_test_split(X,y, random_state=203)

# 1 Nearest Neigbour

## The cell below contains my code for the implemetation of 1 nearest neighour, the code is written in an OOP style, each function has a comment preceding it, the comments give a small decription of what the function does.
###### The test error rate for ionosphere is 0.15909090909090906
###### The test error rate for Nearest Neigbours is 0, it is always correct given my random state

In [8]:
#co-ord calc
def eucDist(arr1,arr2):
    arr3 = arr1 - arr2#calculate the difference between the array values
    arr3 = (arr3 ** 2)#squaring each value in the array
    arr3 = sum(arr3)#adding all the values together
    arr3 = sqrt(arr3)#sqauare rooting the total of the values
    return arr3 

#The function will find the smallest value in an array
def findMin(distances):
    minimum = distances[0]#set the minimum to be intially 0
    index = 0
    for current in range(len(distances)):
        if distances[current] < minimum:
            minimum = distances[current]
            index = current
    return minimum, index #returns the mimimum value and its index

#the function will return true if the label we predict is equal to the actual label in the data set
def assignLabel(predictedIndex, realIndex,trainSet,y_test):
    predictedLabel = trainSet[predictedIndex]
    trueLabel = y_test[realIndex]
    if predictedLabel == trueLabel:
        return True

#main Nearest Neighbours function, in this function nearest neighbours is performed by calling the other functions 
#within the cell
def NearestNeighbours(X_test,X_train,y_test,y_train):
    distances = np.empty(len(X_train), dtype=float)
    correct = 0
    for i in range(len(X_test)):
        for j in range(len(X_train)):
            distances[j] = eucDist(X_train[j],X_test[i])
        minimumDistance = findMin(distances)
        label = assignLabel(minimumDistance[1],i,y_train,y_test)
        if label:
            correct +=1
    percentCorrect = (correct/len(y_test))*100
    print("the algorithm is correct ",percentCorrect,"% of the time")
    print("the error rate is ", (1-correct/len(y_test)))
NearestNeighbours(XIon_test,XIon_train,yIon_test,yIon_train)
NearestNeighbours(XIris_test,XIris_train,yIris_test,yIris_train)

the algorithm is correct  85.22727272727273 % of the time
the error rate is  0.1477272727272727
the algorithm is correct  97.36842105263158 % of the time
the error rate is  0.02631578947368418


# K Nearest Neigbour

## The cell below contains my code for the implemetation of K nearest neighour, the code is written in an OOP style, each function has a comment preceding it, the comments give a small decription of what the function does.
###### The test error rate for ionosphere is 0.375
###### The test error rate for iris is 0.7105263157894737

In [9]:
#this function will retun the three smallest elements of the distances array
def kMinimumDistances(distances):
    indexes = []
    for n in range(len(distances)):
        indexes.append((n,distances[n]))
    for i in range(len(indexes)):
        for j in range(len(indexes)-1):
            if indexes[j][1] > indexes[j+1][1]:
                indexes[j],indexes[j+1] = indexes[j+1], indexes[j]
    return indexes[0:3]

#This method checks to see if the predicted label is equal to the true label
def labelResult(predictedIndex, realIndex,trainSet,y_test):
    predictedLabel = trainSet[predictedIndex]
    trueLabel = y_test[realIndex]
    if predictedLabel == trueLabel:
        return True

#this function will return the label which appear most frequently in the array
def majority(kLabel,y_train):
    labels = []
    for i in range(len(kLabel)):
        index = kLabel[i][0]
        currentLabel = y_train[index]
        labels.append(currentLabel)
    occurence_count = Counter(labels) 
    return occurence_count.most_common(1)[0][0]

#this is the main KNN, from here the algorithm is implemented. It uses all of the methods in the rest of the cell.
def kNearestNeighbours(X_test,X_train,y_test,y_train):
    distances = np.empty(len(X_train), dtype=float)
    correct = 0
    for i in range(len(X_test)):
        for j in range(len(X_train)):
            distances[j] = eucDist(X_train[j],X_test[i])
        minimumDistances = kMinimumDistances(distances)
        label = majority(minimumDistances,y_train)
        result = labelResult(label,i,y_train,y_test)
        if result:
            correct +=1
    percentCorrect = (correct/len(y_test))*100
    print("the algorithm is correct ",percentCorrect,"% of the time")
    print("the error rate is ", (1-correct/len(y_test)))

kNearestNeighbours(XIon_test,XIon_train,yIon_test,yIon_train)
kNearestNeighbours(XIris_test,XIris_train,yIris_test,yIris_train)

the algorithm is correct  81.81818181818183 % of the time
the error rate is  0.18181818181818177
the algorithm is correct  34.21052631578947 % of the time
the error rate is  0.6578947368421053


# Conformal Prediction
## Given below how far I got during my attempts to implement conformal prediction

In [10]:
#this method will return the labels of the current dataset being analysed
def getAllLabels(y_train):
    labels = []
    for i in range(len(y_train)):
        if not (y_train[i] in labels):
            labels.append(y_train[i])
    return labels

#this function will be used to sort the lists of distances for the conformal prediction algorithm. 
def sort(array):
    for i in range(len(array)):
        for j in range(len(array)-1):
            if array[j] > array[j+1]:
                array[j],array[j+1] = array[j+1], array[j]
    return array

#this method will return the distance of the nearest sample of a different label
def findNearestDifferent(label,distances,joinedArray):
    differentLabels = np.where(joinedArray != label)
    minimum = differentLabels[0][0]#set the minimum to be intially 0
    index = 0
    for current in range(len(distances)):
        if (distances[current] < minimum) and (joinedArray[current]!= label):
            minimum = distances[current]
    return minimum
     
#this method will return the distance of the nearest sample of the same label
def findNearestSame(label,distances,joinedArray):
    differentLabels = np.where(joinedArray != label)
    minimum = differentLabels[0][0]#set the minimum to be intially 0
    index = 0
    for current in range(len(distances)):
        if (distances[current] < minimum) and (joinedArray[current]== label):
            minimum = distances[current]
    return minimum

#this method will get the rank of our test sample value 
def findRank(scores, testValue):
    sortedScores = sort(scores)
    rank = np.where(scores==testValue)
    return rank[0]
#the method will get the confrmity score for a given test value. 
def getConformityScore(array,label):
    values = np.empty(len(array), dtype=float)
    conformityScores = np.empty(len(array), dtype=float)
    for a in range(len(array)):
        for b in range(len(array)):
            values[a] = eucDist(array[a],array[b])
        same = findNearestSame(label,values,array)
        different = findNearestDifferent(label, values,array)
        score = different/same
        conformityScores = np.append(conformityScores,score)
    return conformityScores         

#this is the conformal prediction main method. 
def ConformalPrediction(X_test,X_train,y_test,y_train):
    lengths = np.empty(len(X_train)+1, dtype=float)
    correct = 0
    labels = getAllLabels(y_test)
    for i in range(len(X_test)):
        joinedSampleArray = np.vstack((X_train, X_test[i]))
        joinedTestArray = np.append(y_train, y_test[i])
        for k in range(len(labels)):
            lengths = getConformityScore(joinedSampleArray,labels[k])
            #different = findNearestDifferent(labels[k],distances,joinedTestArray)
            #if different == 0:
            #    conformityScore = 0
            ##same = findNearestSame(labels[k],distances,joinedTestArray)
            #if same == 0:
            #    conformityScore = math.inf
            #conformityScore = different/same
            #print(conformityScore)
            pass
        
#ConformalPrediction(XIris_test,XIris_train,yIris_test,yIris_train)
#ConformalPrediction(XIon_test,XIon_train,yIon_test,yIon_train)